# Lesson 28 - Pipelines

## Prepare Environment

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.classification import LogisticRegression 
from pyspark.ml import Pipeline

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.getOrCreate()

## Introduction

In this lesson, we will explore how to using the Spark `Pipeline` class to combine several preprocessing and modeling objects, or **stages**, into a single object.

## Load and Explore Data

To illustrate the use of Pipelines, we will use the [Census Income Dataset](http://archive.ics.uci.edu/ml/datasets/Census+Income) from the UCI Machine Learning Repository.

In [0]:
census_schema = (
    'age BYTE, workclass STRING, fnlwgt INTEGER, education STRING, educ_num BYTE, '
    'marital_status STRING, occupation STRING, relationship STRING, race STRING, '
    'sex STRING, capital_gain INTEGER, capital_loss SHORT, hrs_per_week BYTE, '
    'native_country STRING, salary STRING'
)

census = (
    spark.read
    .option('delimiter', '\t')
    .option('header', True)
    .schema(census_schema)
    .csv('/FileStore/tables/census.txt')
)

census.printSchema()

root
-- age: byte (nullable = true)
-- workclass: string (nullable = true)
-- fnlwgt: integer (nullable = true)
-- education: string (nullable = true)
-- educ_num: byte (nullable = true)
-- marital_status: string (nullable = true)
-- occupation: string (nullable = true)
-- relationship: string (nullable = true)
-- race: string (nullable = true)
-- sex: string (nullable = true)
-- capital_gain: integer (nullable = true)
-- capital_loss: short (nullable = true)
-- hrs_per_week: byte (nullable = true)
-- native_country: string (nullable = true)
-- salary: string (nullable = true)

In [0]:
census.sample(withReplacement=False, fraction=0.0004, seed=1).toPandas()

Out[3]:

,age,workclass,fnlwgt,education,educ_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hrs_per_week,native_country,salary
0,43,Private,154538,Assoc-acdm,12,Divorced,Craft-repair,Not-in-family,White,Male,0,0,40,United-States,<=50K
1,38,Private,65466,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,50,United-States,<=50K
2,35,Self-emp-not-inc,111095,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,<=50K
3,43,Self-emp-not-inc,33521,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Male,0,0,70,United-States,>50K
4,38,Private,87556,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,7688,0,55,United-States,>50K
5,69,Private,277588,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,10,United-States,<=50K
6,58,Self-emp-not-inc,140729,Bachelors,13,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,35,United-States,<=50K
7,62,None,378239,Masters,14,Married-civ-spouse,None,Husband,White,Male,0,0,2,United-States,>50K
8,45,Private,55720,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K
9,33,Local-gov,194901,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,>50K


In [0]:
N = census.count()
print(N)

32561

### Distribution of Label Values

To serve as a baseline against which we can compare our model, we will check the distribution of the label values.

In [0]:
(
    census
    .select('salary')
    .groupby('salary')
    .agg(
        expr('COUNT(*) as count'), 
        expr(f'ROUND(COUNT(*)/{N},4) as prop')
    )
    .show()
)

+------+-----+------+
salary|count| prop|
+------+-----+------+
 <=50K|24720|0.7592|
 >50K| 7841|0.2408|
+------+-----+------+

### Missing Values

We saw in a previous lesson that this dataset contains missing values. We will now determine the number of missing values in each column.

In [0]:
census.select([expr(f'{N} - COUNT({c}) AS {c}') for c in census.columns]).toPandas()

Out[6]:

,age,workclass,fnlwgt,education,educ_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hrs_per_week,native_country,salary
0,0,1836,0,0,0,0,1843,0,0,0,0,0,0,583,0


Before providing this data to an ML model, we need to either remove the records with missing values, or we need to fill in (or impute) the missing values. We will adopt the second approach.

In [0]:
census = census.fillna('missing')

### Identify Numerical and Categorical Features

We need to separate the categorical and numerical features so that we can perform one-hot encoding on the categorical features.

** - another way is to loop over the columns, store any numerical type column into num_features, and string columns into cat_features

In [0]:
num_features = ['age', 'fnlwgt', 'educ_num', 'capital_gain', 'capital_loss', 'hrs_per_week']
cat_features = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

In [0]:
census.columns

Out[9]: ['age',
 'workclass',
 'fnlwgt',
 'education',
 'educ_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hrs_per_week',
 'native_country',
 'salary']

In [0]:
census.dtypes

Out[10]: [('age', 'tinyint'),
 ('workclass', 'string'),
 ('fnlwgt', 'int'),
 ('education', 'string'),
 ('educ_num', 'tinyint'),
 ('marital_status', 'string'),
 ('occupation', 'string'),
 ('relationship', 'string'),
 ('race', 'string'),
 ('sex', 'string'),
 ('capital_gain', 'int'),
 ('capital_loss', 'smallint'),
 ('hrs_per_week', 'tinyint'),
 ('native_country', 'string'),
 ('salary', 'string')]

#### Another way of avoiding hard coding to identify the numerical dtype

In [0]:
cat_features = [ c for c, d in census.dtypes[:-1] if d == 'string']  # slice out the last column, which is the target
num_features = [c for c in census.columns[:-1] if c not in cat_features]
# num_features = [c for c in census.dtypes in d != 'string'] this is another solution for identifying numerical features

print(cat_features)
print(num_features)

['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']
['age', 'fnlwgt', 'educ_num', 'capital_gain', 'capital_loss', 'hrs_per_week']

## Create and Apply the Pipeline

### Define Pipeline Stages

We will now define several stages to be included within our pipeline object. Each of these stages represents either a preprocessing task to be performed on the data, or the logistic regression model itself.

In [0]:
ix_features = [c + '_ix' for c in cat_features]
vec_features = [c + '_vec' for c in cat_features]

label_indexer = StringIndexer(inputCol='salary', outputCol='label')

feature_indexer = StringIndexer(inputCols=cat_features, outputCols=ix_features)

encoder = OneHotEncoder(inputCols=ix_features, outputCols=vec_features, dropLast=False)

assembler = VectorAssembler(inputCols=num_features + vec_features, outputCol='features')

logreg = LogisticRegression(featuresCol='features', labelCol='label')

### Create and Fit the Pipeline

We will now use the stages to create a Pipeline object, which we will then fit to the census data. We will use the fitted pipeline to generate predictions for the training set.

In [0]:
model = Pipeline(stages=[label_indexer, feature_indexer, encoder, assembler, logreg]).fit(census)
train_pred = model.transform(census)

train_pred.select(['probability', 'prediction', 'label']).show(10, truncate=False)

+----------------------------------------+----------+-----+
probability |prediction|label|
+----------------------------------------+----------+-----+
[0.8709980132804843,0.12900198671951574]|0.0 |0.0 |
[0.5973627766226276,0.40263722337737246]|0.0 |0.0 |
[0.9717847458770827,0.02821525412291721]|0.0 |0.0 |
[0.9178280523465923,0.08217194765340777]|0.0 |0.0 |
[0.32724389387742414,0.6727561061225759]|1.0 |0.0 |
[0.17321265218788923,0.8267873478121107]|1.0 |0.0 |
[0.9982895628832512,0.00171043711674872]|0.0 |0.0 |
[0.6049360234198429,0.39506397658015713]|0.0 |1.0 |
[0.09284776707198127,0.9071522329280188]|1.0 |1.0 |
[0.08311365578619417,0.9168863442138058]|1.0 |1.0 |
+----------------------------------------+----------+-----+
only showing top 10 rows

### Score the Model

We will calculate the model's accuracy on the training set.

In [0]:
accuracy_eval = MulticlassClassificationEvaluator(
    predictionCol='prediction', labelCol='label', metricName='accuracy')

acc = accuracy_eval.evaluate(train_pred)
print(acc)

0.8533521697736556

### Generate Predictions for New Data

Next we will apply our model to new observations. Notice that since the pipeline with have created contains all of the preprocessing stages, we don't need to apply these separately to the new dataset.

In [0]:
new_schema = (
    'age BYTE, workclass STRING, fnlwgt INTEGER, education STRING, educ_num BYTE, '
    'marital_status STRING, occupation STRING, relationship STRING, race STRING, '
    'sex STRING, capital_gain INTEGER, capital_loss SHORT, hrs_per_week BYTE, '
    'native_country STRING'
)

new_data = spark.createDataFrame(
    data = [[53, 'Private', 94081, 'Bachelors', 13, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Male', 0, 0, 44, 'United-States'], 
            [21, 'Private', 202214, 'Assoc-acdm', 12, 'Never-married', 'Adm-clerical', 'Not-in-family', 'White', 'Female', 0, 0, 10, 'United-States'], 
            [40, 'Local-gov', 33155, 'HS-grad', 9, 'Never-married', 'Other-service', 'Own-child', 'Black', 'Male', 0, 0, 40, 'United-States'], 
            [44, 'Private', 147206, 'Some-college', 10, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'White', 'Female', 0, 0, 40, 'United-States'], 
            [48, 'Local-gov', 493862, 'HS-grad', 9, 'Married-civ-spouse', 'Exec-managerial', 'Husband', 'Black', 'Male', 7298, 0, 38, 'United-States']],
    schema = new_schema
)

new_pred = model.transform(new_data)

new_pred.select(['probability', 'prediction']).show(truncate=False)

+------------------------------------------+----------+
probability |prediction|
+------------------------------------------+----------+
[0.24945659959217034,0.7505434004078296] |1.0 |
[0.9934231400941665,0.006576859905833343] |0.0 |
[0.9970888534414106,0.0029111465585892797]|0.0 |
[0.7002063180345515,0.2997936819654485] |0.0 |
[0.12749152013502485,0.8725084798649753] |1.0 |
+------------------------------------------+----------+

## Stages

In [0]:
for stage in model.stages:
    print(stage)

StringIndexerModel: uid=StringIndexer_924c91242e7e, handleInvalid=error
StringIndexerModel: uid=StringIndexer_eae376ccab45, handleInvalid=error, numInputCols=8, numOutputCols=8
OneHotEncoderModel: uid=OneHotEncoder_01817f9b9a03, dropLast=false, handleInvalid=error, numInputCols=8, numOutputCols=8
VectorAssembler_0c2eb2e38618
LogisticRegressionModel: uid=LogisticRegression_f0b0c193ba71, numClasses=2, numFeatures=108

In [0]:
# I want to know the encoding for my label

print(model.stages[0].labels)

['<=50K', '>50K']

In [0]:
# I want to know the string indexer
for c, labels in zip(cat_features, model.stages[1].labelsArray):
    print(c)
    print(labels)
    print()

workclass
('Private', 'Self-emp-not-inc', 'Local-gov', 'missing', 'State-gov', 'Self-emp-inc', 'Federal-gov', 'Without-pay', 'Never-worked')

education
('HS-grad', 'Some-college', 'Bachelors', 'Masters', 'Assoc-voc', '11th', 'Assoc-acdm', '10th', '7th-8th', 'Prof-school', '9th', '12th', 'Doctorate', '5th-6th', '1st-4th', 'Preschool')

marital_status
('Married-civ-spouse', 'Never-married', 'Divorced', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse')

occupation
('Prof-specialty', 'Craft-repair', 'Exec-managerial', 'Adm-clerical', 'Sales', 'Other-service', 'Machine-op-inspct', 'missing', 'Transport-moving', 'Handlers-cleaners', 'Farming-fishing', 'Tech-support', 'Protective-serv', 'Priv-house-serv', 'Armed-Forces')

relationship
('Husband', 'Not-in-family', 'Own-child', 'Unmarried', 'Wife', 'Other-relative')

race
('White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other')

sex
('Male', 'Female')

native_country
('United-States', 'Mexico', 'missing', 'Philippines', 'Germany', 'Canada', 'Puerto-Rico', 'El-Salvador', 'India', 'Cuba', 'England', 'Jamaica', 'South', 'China', 'Italy', 'Dominican-Republic', 'Vietnam', 'Guatemala', 'Japan', 'Poland', 'Columbia', 'Taiwan', 'Haiti', 'Iran', 'Portugal', 'Nicaragua', 'Peru', 'France', 'Greece', 'Ecuador', 'Ireland', 'Hong', 'Cambodia', 'Trinadad&Tobago', 'Laos', 'Thailand', 'Yugoslavia', 'Outlying-US(Guam-USVI-etc)', 'Honduras', 'Hungary', 'Scotland', 'Holand-Netherlands')

In [0]:
print(model.stages[-1].coefficients) # coefficients for all the features

[0.025520953489476424,7.076673516286993e-07,-0.019273639003916536,0.0003191556055164523,0.000647353338493932,0.029713515088417557,-0.30833155544617175,-0.799029305860313,-0.4909073433876235,-0.622921145196694,-0.6191506776851085,-0.13330826723822428,0.19403015747465102,-7.088027745446063,-5.066917784691313,-0.5928224336039796,-0.22799630000447685,0.6071331961613919,0.9791696827289731,-0.015561636399226973,-1.3594961483941213,-0.012828103628768026,-1.464061770781904,-1.9789633863848133,1.5027199898602748,-1.677261602366024,-0.9359884399170212,1.7183247384415665,-1.7601642315549486,-2.0752816396124194,-18.274783447883564,1.213024118630004,-1.474576933502891,-0.9918070789371819,-1.1247199370622554,-0.8637092832536373,-1.0027715788320706,1.692429138228219,0.2634657031510486,-0.17990788286545595,0.5348471439856354,-0.25199695109928316,0.029312771652159925,-1.0772509824206482,-0.5339351422514415,-0.6410918065742872,-0.36151268487530297,-0.9305693818719527,-1.2441626413857443,0.40413398615178014,0.3248947781879807,-4.370181883375429,-1.420300590294452,-0.4457124217992475,0.12225306190502955,-1.1056631301490547,-0.005883401494451053,0.9169543182442824,-0.8193147633475004,-0.733484145520854,-0.9124610045111046,-0.6409866777698343,-1.3063806918973766,-1.132861467469389,-0.04799240354998029,-0.9095290586050918,-0.833778534487571,-1.580104369396258,-1.215441416208552,-0.6054304452160983,-0.5958562349942667,-0.698401420666663,-1.3640319175355065,-1.6372035188128111,-1.4052026595123683,-0.6814107780741214,-0.7199627785891339,-0.9866567585441385,-2.09763746199113,-1.7237541383917,-0.2209234233319616,-2.858001664402637,-2.1749964477932324,-1.2788587110112097,-0.636439076794724,-1.0331248802451263,-3.147902972093633,-0.9910946673253329,-1.078964795126654,-0.9814304034257114,-1.060795238666226,-1.8300033982967485,-1.8650124291112573,-0.4424120158709269,-2.0127862722185808,-1.3100795140082402,-0.4952295389357556,-1.1288480820270506,0.26607605726766936,-1.412314331658438,-1.636983709570417,-1.5950286030956031,-0.3433055564489484,-7.183780167894288,-2.3071295540266386,-1.1426150690454633,-1.0247522479617477,-5.256204310958342]

### Get all the features coefficient

In [0]:
final_features = num_features.copy()

for labarray in model.stages[1].labelsArray:
    final_features += list(labarray)

In [0]:
pd.set_option('max_rows', None)

In [0]:
pd.DataFrame({
    'feature': final_features,
    'coefficient': model.stages[-1].coefficients
}
)

Out[46]:

,feature,coefficient
0,age,2.552095e-02
1,fnlwgt,7.076674e-07
2,educ_num,-1.927364e-02
3,capital_gain,3.191556e-04
4,capital_loss,6.473533e-04
5,hrs_per_week,2.971352e-02
6,Private,-3.083316e-01
7,Self-emp-not-inc,-7.990293e-01
8,Local-gov,-4.909073e-01
9,missing,-6.229211e-01
